In [42]:
import starepandas
import pickle
import sqlalchemy
import pandas
import numpy
import datetime
import time

In [16]:
level = 10

# Load Cover

In [17]:
fname = 'covers/caribbean_sids_cover_l{}.pickle'.format(level)
with open(fname, 'rb') as f:
    cover_sids = pickle.load(f)

# Load catalogue

In [18]:
db_path = 'catalogues/catalogue.sqlite' 
uri = 'sqlite:///{db_path}'.format(db_path=db_path)
engine = sqlalchemy.create_engine(uri)

In [19]:
catalogue = starepandas.read_sql_table(sql='cldmsk_noaa_20', con=engine, stare_column='stare_cover')

# Find nonintersecting

In [20]:
intersects = catalogue.stare_intersects(cover_sids, n_workers=50)

In [21]:
# Those guys dont intersect our ROI
catalogue[intersects==False]

,begining,ending,granule_path,sidecar_path,stare_cover
2,2018-03-27T06:42:00.000Z,2018-03-27T06:48:00.000Z,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,"[2675138178658074627, 2693152577167556611, 266..."
4,2019-08-28T04:24:00.000Z,2019-08-28T04:30:00.000Z,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,"[2558044588346441731, 2494994193563254788, 249..."
5,2020-05-23T08:48:00.000Z,2020-05-23T08:54:00.000Z,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,"[2972375754064527363, 2981382953319268355, 292..."
6,2019-01-06T05:42:00.000Z,2019-01-06T05:48:00.000Z,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,"[3035426148847714307, 3602879701896396803, 244..."
8,2019-01-28T05:30:00.000Z,2019-01-28T05:36:00.000Z,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,"[3602879701896396803, 2449958197289549828, 245..."
...,...,...,...,...,...
9284,2020-12-20T07:36:00.000Z,2020-12-20T07:42:00.000Z,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,"[3035426148847714307, 3080462145121419267, 308..."
9287,2019-03-14T08:24:00.000Z,2019-03-14T08:30:00.000Z,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,"[2990390152574009347, 3017411750338232323, 292..."
9288,2018-08-28T06:54:00.000Z,2018-08-28T07:00:00.000Z,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,"[2675138178658074627, 2936346957045563395, 294..."
9292,2019-07-24T08:48:00.000Z,2019-07-24T08:54:00.000Z,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/C...,"[2972375754064527363, 2981382953319268355, 292..."


In [30]:
runtime = []
level = 10

In [31]:
fname = 'covers/caribbean_sids_cover_l{}.pickle'.format(level)
with open(fname, 'rb') as f:
    cover_sids = pickle.load(f)

In [32]:
catalogue = starepandas.read_sql_table(sql='cldmsk_noaa_20', con=engine, stare_column='stare_cover')

In [45]:
start = datetime.datetime.now()
intersects = catalogue.stare_intersects(cover_sids, n_workers=50)
catalogue[intersects==False]
total = datetime.datetime.now() - start
total
#runtime.append('Level:' + level + 'Runtime:' +  time )

datetime.timedelta(0, 21, 787039)

In [ ]:
catalogue[intersects==False]

In [34]:
fname = 'covers/caribbean_sids_cover_l{}.pickle'
sql='cldmsk_noaa_20'
level = 10
n_workers = 60

In [46]:
def runtime(fname, sql, level, n_workers):
    fname = fname.format(level)
    with open(fname, 'rb') as f:
        cover_sids = pickle.load(f)
    catalogue = starepandas.read_sql_table(sql=sql, con=engine, stare_column='stare_cover')
    start = time.time()
    intersects = catalogue.stare_intersects(cover_sids, n_workers=n_workers)
    catalogue[intersects==False]
    #time = datetime.datetime.now() - start
    return(level, time.time() - start)
    #return('Level:' + level + 'Runtime:' + time)

In [54]:
runtime(fname, sql, 10, n_workers)

(10, 20.529750108718872)